In [1]:
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
mitosis_dir = 'softec-aic-2024/Training Images/Mitosis'
normal_dir = 'softec-aic-2024/Training Images/Normal'

img_size = (64, 64)

X = []
Y = []

for i in range(0, 226):
    filename = str(i) + '.jpg'

    if filename in os.listdir(mitosis_dir):
        label = 1
    else:
        label = 0

    img = Image.open(os.path.join(mitosis_dir if label else normal_dir, filename))
    img = img.resize(img_size)

    img_array = np.array(img) / 255.0

    X.append(img_array)
    Y.append(label)

In [3]:
X = np.array(X)
Y = np.array(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [4]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPool2D())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [5]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
history = model.fit(X, Y, epochs=20, batch_size=32)

Epoch 1/20


8/8 [==============================] - 7s 89ms/step - loss: 0.8156 - accuracy: 0.5442
Epoch 2/20
8/8 [==============================] - 1s 77ms/step - loss: 0.6395 - accuracy: 0.5885
Epoch 3/20
8/8 [==============================] - 1s 72ms/step - loss: 0.5520 - accuracy: 0.7522
Epoch 4/20
8/8 [==============================] - 1s 88ms/step - loss: 0.4643 - accuracy: 0.7965
Epoch 5/20
8/8 [==============================] - 1s 76ms/step - loss: 0.4882 - accuracy: 0.8142
Epoch 6/20
8/8 [==============================] - 1s 81ms/step - loss: 0.4534 - accuracy: 0.8230
Epoch 7/20
8/8 [==============================] - 1s 81ms/step - loss: 0.4490 - accuracy: 0.8097
Epoch 8/20
8/8 [==============================] - 1s 75ms/step - loss: 0.5194 - accuracy: 0.7478
Epoch 9/20
8/8 [==============================] - 1s 82ms/step - loss: 0.5395 - accuracy: 0.7743
Epoch 10/20
8/8 [==============================] - 1s 101ms/step - loss: 0.4470 - accuracy: 0.8540
Epoch 11/20
8/8 [=========

In [7]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

2/2 [==============================] - 1s 36ms/step - loss: 0.4548 - accuracy: 0.8478
Loss: 0.4547933042049408, Accuracy: 0.8478260636329651


# Testing

In [10]:
test_images = []
for i in range(226,283):
    filename = str(i) + '.jpg'
    img = Image.open(os.path.join('softec-aic-2024/Testing Images', filename))
    img = img.resize(img_size)
    img_array = np.array(img) / 255.0
    test_images.append(img_array)

test_images = np.array(test_images)

In [11]:
predictions = model.predict(test_images)

2/2 [==============================] - 1s 31ms/step


In [13]:
test = pd.read_csv('softec-aic-2024/Test.csv')
test['Label'] = predictions
test['Label'] = test['Label'].apply(lambda x: 'Mitosis' if x > 0.5 else 'Normal')

In [15]:
test.to_csv('predtions.csv',index=False)